## Running C code from Jupyter

In [3]:
import os
from ctypes import *

In [4]:
# Shared object that can be unloaded
# WARNING: OS dependent! (written for Linux)
class SharedObjectInterface(object):
    # https://stackoverflow.com/questions/359498/how-can-i-unload-a-dll-using-ctypes-in-python
    
    def __init__(self, path):
        self.path = path
        self.lib = CDLL(path)
    
    def close(self):
        while self._is_loaded():
            self._dlclose()
        
    def _is_loaded(self):
        libp = os.path.abspath(self.path)
        ret = os.system("lsof -p %d | grep %s > /dev/null" % (os.getpid(), libp))
        return (ret == 0)

    def _dlclose(self):
        libdl = CDLL("libdl.so")
        libdl.dlclose(self.lib._handle)


In [5]:
%%writefile vcf.c
#include <stdint.h>

/*
def nb_onepole(x, g, x1, y1):
    v = 1/1.3 * x + 0.3/1.3*x1
    y = (v-y1)*g + y1
    return y
*/
static float onepole(float x, float g, float x1, float y1)
{
    float v = 1.0/1.3 * x + 0.3/1.3*x1;
    return (v-y1)*g + y1;
}

/*
def nb_vcf_linear(x, g, k):
    # Onepole delay points:
    # - Before op1
    # - After opN for N in 1...4
    y = [0]*len(x)
    w = [0, 0, 0, 0, 0] # Current sample
    d = [0, 0, 0, 0, 0] # Delayed sample
    
    # Feedback gain compensation form Valimaki
    G_comp = 0.5

    for n in range(1, len(y)):     
        x_ = x[n]
        w[0] = x_ - (d[4]*k - G_comp*x_)
        w[1] = nb_onepole(w[0], g, d[0], d[1])
        w[2] = nb_onepole(w[1], g, d[1], d[2])
        w[3] = nb_onepole(w[2], g, d[2], d[3])
        w[4] = nb_onepole(w[3], g, d[3], d[4])

        for i in range(5):
            d[i] = w[i]

        # Limit resonance peaking
        y[n] = 1/8 * w[4]

    return y
*/
void vcf(float *x, float *y, int32_t length, float g, float k)
{
    int n, i;
    float x_;
    float w[5] = {0, 0, 0, 0, 0}; // Current sample
    float d[5] = {0, 0, 0, 0, 0}; // Delayed sample
    float G_comp = 0.5;
    
    for (n = 0; n < length; n++) {
        // Limit resonance peaks
        x_ = x[n];
        w[0] = x_ - (d[4]*k - G_comp*x_);
        w[1] = onepole(w[0], g, d[0], d[1]);
        w[2] = onepole(w[1], g, d[1], d[2]);
        w[3] = onepole(w[2], g, d[2], d[3]);
        w[4] = onepole(w[3], g, d[3], d[4]);
        
        for (i = 0; i < 5; i++) {
            d[i] = w[1];
        }
        
        y[n] = w[4];
    }
}

Overwriting vcf.c


In [6]:
! gcc -shared -Wall -o vcf.so -fPIC vcf.c

In [9]:
class VcfInterface(SharedObjectInterface):
    def __init__(self, path):
        super().__init__(path)
        self.vcf_ = self.lib.vcf
        self.vcf_.argtypes = [c_void_p, c_void_p]
        
    def vcf(self, x, g, k):
        x_ = (c_float*len(x))(*x)
        y_ = (c_float*len(x))()
        self.vcf_(byref(x_), byref(y_), len(x_), c_float(g), c_float(k))
        return [X for X in y_]

try:
    c_vcf_so.close()
finally:
    c_vcf_so = VcfInterface("./vcf.so")


       

In [12]:
x = list(range(1000))
y = c_vcf_so.vcf(x, 0.5, 2.0)